In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader,Dataset
from model_classes.convNet import ConvNet
#import torchvision.datasets.ImageFolder 
import matplotlib.pyplot as plt
import numpy as np
import os
from const import *
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        #print(f"image path: {img_path}")
        image = read_image(img_path)
        image=image.to(torch.float32)
        #print(f"image type: {type(image)}")
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            #print('here')
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
train_dataset= torchvision.datasets.ImageFolder(TRAIN_IMAGE_PATH,transform)
test_dataset= torchvision.datasets.ImageFolder(TEST_IMAGE_PATH,transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=True)




In [3]:
print(train_dataset[0][0].size())

torch.Size([3, 44, 6])


In [7]:
def train(model,num_epochs=1000,learning_rate=0.001):

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        
    n_total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            # origin shape: [4, 3, 32, 32] = 4, 3, 1024
            # input_layer: 3 input channels, 6 output channels, 5 kernel size
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 200 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

    print('Finished Training')
    PATH = './cnn.pth'
    torch.save(model.state_dict(), PATH)

In [8]:
#Load model and train
output_size=10
model = ConvNet(output_size=10)
MODEL_PATH=os.path.join("trained_models","28_12.pth")
model.load_state_dict(torch.load(MODEL_PATH,map_location=torch.device(device)))
model.eval()
train(model)

Epoch [1/1000], Step [200/314], Loss: 0.0049
Epoch [2/1000], Step [200/314], Loss: 1.7183
Epoch [3/1000], Step [200/314], Loss: 1.0908
Epoch [4/1000], Step [200/314], Loss: 0.0284
Epoch [5/1000], Step [200/314], Loss: 0.0454
Epoch [6/1000], Step [200/314], Loss: 0.0544
Epoch [7/1000], Step [200/314], Loss: 0.0324
Epoch [8/1000], Step [200/314], Loss: 0.2127
Epoch [9/1000], Step [200/314], Loss: 0.8310
Epoch [10/1000], Step [200/314], Loss: 2.6334
Epoch [11/1000], Step [200/314], Loss: 0.0809
Epoch [12/1000], Step [200/314], Loss: 0.0384
Epoch [13/1000], Step [200/314], Loss: 0.0018
Epoch [14/1000], Step [200/314], Loss: 0.1986
Epoch [15/1000], Step [200/314], Loss: 0.0403
Epoch [16/1000], Step [200/314], Loss: 0.1611
Epoch [17/1000], Step [200/314], Loss: 0.0054
Epoch [18/1000], Step [200/314], Loss: 0.0163
Epoch [19/1000], Step [200/314], Loss: 0.0209
Epoch [20/1000], Step [200/314], Loss: 0.0346
Epoch [21/1000], Step [200/314], Loss: 0.4175
Epoch [22/1000], Step [200/314], Loss: 0.59

In [4]:
#Train from scratch
output_size=10
print(device)
num_epochs = 1000
learning_rate = 0.001
model = ConvNet(output_size=output_size).to(device)
model.eval()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 200 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(output_size)]
    n_class_samples = [0 for i in range(output_size)]
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

cpu
Epoch [1/1000], Step [200/213], Loss: 2.3784
Epoch [2/1000], Step [200/213], Loss: 2.3691
Epoch [3/1000], Step [200/213], Loss: 2.2533
Epoch [4/1000], Step [200/213], Loss: 2.3215
Epoch [5/1000], Step [200/213], Loss: 2.3043
Epoch [6/1000], Step [200/213], Loss: 2.2934
Epoch [7/1000], Step [200/213], Loss: 2.3156
Epoch [8/1000], Step [200/213], Loss: 2.2898
Epoch [9/1000], Step [200/213], Loss: 2.3389
Epoch [10/1000], Step [200/213], Loss: 2.2555
Epoch [11/1000], Step [200/213], Loss: 2.2748
Epoch [12/1000], Step [200/213], Loss: 2.2549
Epoch [13/1000], Step [200/213], Loss: 2.3905
Epoch [14/1000], Step [200/213], Loss: 2.3130
Epoch [15/1000], Step [200/213], Loss: 2.2786
Epoch [16/1000], Step [200/213], Loss: 2.2962
Epoch [17/1000], Step [200/213], Loss: 2.2967
Epoch [18/1000], Step [200/213], Loss: 2.2974
Epoch [19/1000], Step [200/213], Loss: 2.2276
Epoch [20/1000], Step [200/213], Loss: 2.2074
Epoch [21/1000], Step [200/213], Loss: 2.2819
Epoch [22/1000], Step [200/213], Loss: 